In [ ]:
import os
import shutil
import joblib
from pathlib import Path
import glob
import subprocess
import time
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import PCA
import shapeworks as sw

# Pour tracer et insérer des images
import matplotlib.pyplot as plt
from openpyxl.drawing.image import Image as ExcelImage
import tempfile



  ### 0. Paramètres

In [ ]:
# ---------- Dossiers ----------
DATASET_NAME       = "TEST"
DATASET_PATHS      = [
    #(os.path.join(".", "DATA", "RF_FULGUR_M"), "RF"),
    (os.path.join(".", "DATA", "RF_FULGUR_SAMPLE_2"), "RF_S2") #pour tester
]
SHAPE_EXT          = ".nii.gz"

# Les sorties seront placées dans "./OUTPUT_PIPELINE"
BASE_OUTPUT_DIR    = os.path.abspath(os.path.join(".", "OUTPUT_PIPELINE"))
OPTION_SET         = "default"  # Option set à utiliser dans les noms de fichiers

# ---------- Paramètres Grooming ----------
ANTIALIAS_ITERATIONS = 30
ISO_SPACING          = [1, 1, 1]
PAD_SIZE             = 10     
PAD_VALUE            = 0
ISO_VALUE            = 0.5
ICP_ITERATIONS       = 100

# ---------- Paramètres par Défaut ----------
OPT_PARAMS = {
    "number_of_particles":       128,
    "use_normals":               0,
    "normals_strength":          10.0,
    "checkpointing_interval":    1000,
    "keep_checkpoints":          0,
    "iterations_per_split":      1000,
    "optimization_iterations":   1000,
    "starting_regularization":   10,
    "ending_regularization":     1,
    "relative_weighting":        1,
    "initial_relative_weighting":0.05,
    "procrustes_interval":       0,
    "procrustes_scaling":        0,
    "save_init_splits":          0,
    "verbosity":                 0,
    "multiscale":                1,
    "multiscale_particles":      32,
    "tiny_test":                 False,
    "use_single_scale":          0,
    "mesh_mode":                 True,
    "option_set":                OPTION_SET
}

# ---------- Paramètres Variables (pour grid search) ----------
GRID_PARAMS = {
    "number_of_particles": [16, 32]
}

In [ ]:
OUTPUT_DB = os.path.abspath(os.path.join(".", "OUTPUT_DB"))
if not os.path.exists(OUTPUT_DB):
    os.makedirs(OUTPUT_DB)

# Si le dossier de sortie existe, déplacer les fichiers Excel qu'il contient
if os.path.exists(BASE_OUTPUT_DIR):
    for root, dirs, files in os.walk(BASE_OUTPUT_DIR):
        for file in files:
            if file.endswith(".xlsx"):
                source_file = os.path.join(root, file)
                # Créer un nom unique en ajoutant le timestamp
                new_filename = f"{os.path.splitext(file)[0]}_{int(time.time())}.xlsx"
                dest_file = os.path.join(OUTPUT_DB, new_filename)
                shutil.move(source_file, dest_file)
    # Supprimer entièrement le dossier de sortie pour repartir d'une base propre
    shutil.rmtree(BASE_OUTPUT_DIR)


  ### 1. Grooming

In [ ]:
def acquire_data(DATASET_PATHS, SHAPE_EXT, OUTPUT_PATH):
    print("\n----------------------------------------")
    print("Step 1. Acquire Data")
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)
    shape_filenames = []  # Liste des fichiers
    dataset_ids = []      # Identifiants associés
    for data_path, dataset_id in DATASET_PATHS:
        files = sorted(glob.glob(os.path.join(data_path, '*' + SHAPE_EXT)))
        shape_filenames.extend(files)
        dataset_ids.extend([dataset_id] * len(files))
    print(f"Nombre de shapes : {len(shape_filenames)}")
    return shape_filenames, dataset_ids

def groom_shapes(shape_filenames, dataset_ids, groom_dir):
    print("\n----------------------------------------")
    print("Step 2. Groom - Data Pre-processing")
    if not os.path.exists(groom_dir):
        os.makedirs(groom_dir)
    shape_seg_list = []
    shape_names = []
    for i, shape_filename in enumerate(tqdm(shape_filenames, desc="Grooming shapes")):
        dataset_id = dataset_ids[i]
        base_shape_name = os.path.basename(shape_filename).replace(SHAPE_EXT, '')
        shape_name = f"{dataset_id}_{base_shape_name}"
        shape_names.append(shape_name)
        shape_seg = sw.Image(shape_filename)
        shape_seg_list.append(shape_seg)
        bounding_box = sw.ImageUtils.boundingBox([shape_seg], ISO_VALUE).pad(2)
        shape_seg.crop(bounding_box)
        shape_seg.antialias(ANTIALIAS_ITERATIONS).resample(ISO_SPACING, sw.InterpolationType.Linear).binarize()
        shape_seg.pad(PAD_SIZE, PAD_VALUE)
    return shape_seg_list, shape_names

def rigid_transformations(shape_seg_list, shape_names, groom_dir):
    print("\n----------------------------------------")
    print("Step 3. Groom - Rigid Transformations")
    print("Recherche de l'image de référence...")
    ref_index = sw.find_reference_image_index(shape_seg_list)
    ref_seg = shape_seg_list[ref_index]
    ref_name = shape_names[ref_index]
    ref_filename = os.path.join(groom_dir, 'reference.nii.gz')
    ref_seg.write(ref_filename)
    print("Image de référence trouvée : " + ref_name)
    transform_dir = os.path.join(groom_dir, 'rigid_transforms')
    if not os.path.exists(transform_dir):
        os.makedirs(transform_dir)
    rigid_transforms = []
    for shape_seg, shape_name in tqdm(zip(shape_seg_list, shape_names), desc="Calcul des transformations", total=len(shape_seg_list)):
        rigid_transform = shape_seg.createRigidRegistrationTransform(ref_seg, ISO_VALUE, ICP_ITERATIONS)
        rigid_transform = sw.utils.getVTKtransform(rigid_transform)
        rigid_transforms.append(rigid_transform)
        transform_filename = os.path.join(transform_dir, f'{shape_name}_to_{ref_name}_transform.txt')
        np.savetxt(transform_filename, rigid_transform)
        shape_seg.antialias(ANTIALIAS_ITERATIONS).computeDT(0).gaussianBlur(1.5)
    # Détermination du mode de sauvegarde (meshes ou distance_transforms)
    if OPT_PARAMS.get("mesh_mode", True):
        output_subdir = 'meshes'
        output_extension = '.vtk'
    else:
        output_subdir = 'distance_transforms'
        output_extension = SHAPE_EXT
    OUTPUT_DIR = os.path.join(groom_dir, output_subdir)
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
    groomed_files = []
    for shape_seg, shape_name in zip(shape_seg_list, shape_names):
        output_filename = os.path.join(OUTPUT_DIR, shape_name + output_extension)
        shape_seg.write(output_filename)
        groomed_files.append(output_filename)
    return rigid_transforms, groomed_files



  ### 2. Optimisation

In [ ]:
def optimize_particles(shape_seg_list, shape_filenames, rigid_transforms, groomed_files, OUTPUT_PATH):
    """
    Fonction d'optimisation qui crée un projet ShapeWorks et lance shapeworks optimize.
    """
    print("\n----------------------------------------")
    print("Step 4. Optimize - Particle Based Optimization")
    if not os.path.exists(OUTPUT_PATH):
        os.makedirs(OUTPUT_PATH)

    # --- Étape clé : on récupère le domain_type et on ajuste groomed_files
    domain_type, groomed_files = sw.data.get_optimize_input(
        groomed_files,
        OPT_PARAMS["mesh_mode"]  # True si on veut des meshes, False sinon
    )

    # Construction des sujets (1 seul domain par sujet)
    subjects = []
    number_domains = 1
    for i in range(len(shape_seg_list)):
        subject = sw.Subject()
        subject.set_number_of_domains(number_domains)
        # Fichiers originaux
        abs_shape_filename = os.path.abspath(shape_filenames[i])
        subject.set_original_filenames([abs_shape_filename])
        # Fichiers groomed
        groomed_file = os.path.abspath(groomed_files[i])
        subject.set_groomed_filenames([groomed_file])
        # Transformations rigides
        transform = [rigid_transforms[i].flatten()]
        subject.set_groomed_transforms(transform)
        # Pour versions récentes de ShapeWorks
        try:
            subject.set_domain_type(0, domain_type)
        except AttributeError:
            pass
        subjects.append(subject)

    # Création du projet ShapeWorks et ajout des paramètres
    project = sw.Project()
    project.set_subjects(subjects)
    parameters = sw.Parameters()

    # On prépare uniquement les paramètres valides pour l'optimisation
    valid_params = {
        "number_of_particles":        OPT_PARAMS["number_of_particles"],
        "use_normals":                OPT_PARAMS["use_normals"],
        "normals_strength":           OPT_PARAMS["normals_strength"],
        "checkpointing_interval":     OPT_PARAMS["checkpointing_interval"],
        "keep_checkpoints":           OPT_PARAMS["keep_checkpoints"],
        "iterations_per_split":       OPT_PARAMS["iterations_per_split"],
        "optimization_iterations":    OPT_PARAMS["optimization_iterations"],
        "starting_regularization":    OPT_PARAMS["starting_regularization"],
        "ending_regularization":      OPT_PARAMS["ending_regularization"],
        "relative_weighting":         OPT_PARAMS["relative_weighting"],
        "initial_relative_weighting": OPT_PARAMS["initial_relative_weighting"],
        "procrustes_interval":        OPT_PARAMS["procrustes_interval"],
        "procrustes_scaling":         OPT_PARAMS["procrustes_scaling"],
        "save_init_splits":           OPT_PARAMS["save_init_splits"],
        "verbosity":                  OPT_PARAMS["verbosity"]
    }

    if OPT_PARAMS.get("tiny_test", False):
        valid_params["number_of_particles"] = 32
        valid_params["optimization_iterations"] = 25

    # Multiscale ou single scale
    if not OPT_PARAMS.get("use_single_scale", 0):
        valid_params["multiscale"] = 1
        valid_params["multiscale_particles"] = OPT_PARAMS["multiscale_particles"]

    # Remplir les paramètres
    for key, val in valid_params.items():
        parameters.set(key, sw.Variant([val]))
    project.set_parameters("optimize", parameters)

    # Sauvegarde du fichier projet
    proj_filename = os.path.join(OUTPUT_PATH, f"{DATASET_NAME}_{OPTION_SET}.swproj")
    project.save(proj_filename)

    # Lancement de l'optimisation
    print("Lancement de l'optimisation via shapeworks...")
    optimize_cmd = ['shapeworks', 'optimize', '--progress', '--name', proj_filename]
    subprocess.check_call(optimize_cmd, cwd=OUTPUT_PATH)

    # Vérification finale
    args_for_check = type('ArgsForCheck', (object,), {})()
    args_for_check.tiny_test = OPT_PARAMS.get("tiny_test", False)
    args_for_check.verify    = False
    sw.utils.check_results(args_for_check, proj_filename)

    return os.path.join(OUTPUT_PATH, f"{DATASET_NAME}_default_particles")




  ### 3. Analyse en Composantes Principales

In [ ]:
def get_particles(particles_dir, particle_type="world"):
    particles = []
    names = []
    for filename in os.listdir(particles_dir):
        if filename.endswith(particle_type + ".particles"):
            filepath = os.path.join(particles_dir, filename)
            try:
                data = np.loadtxt(filepath)
                particles.append(data)
                names.append(os.path.splitext(filename)[0])
            except Exception as e:
                print(f"Erreur de lecture du fichier {filename}: {e}")
    if particles:
        return np.array(particles), names
    return None, None

def compute_pca(particles_dir, PCA_OUTPUT_DIR):
    print("\n----------------------------------------")
    print("Calcul de l'ACP")
    if not os.path.exists(PCA_OUTPUT_DIR):
        os.makedirs(PCA_OUTPUT_DIR)
    particles, shape_names = get_particles(particles_dir, particle_type="world")
    if particles is None:
        raise ValueError("Aucune particule chargée depuis " + particles_dir)
    particles_flat = particles.reshape(particles.shape[0], -1)
    n, d = particles_flat.shape
    print(f"Forme des particules aplaties : {n} x {d}")
    pca = PCA(n_components=n - 1)
    pca.fit(particles_flat)
    components = pca.transform(particles_flat)
    # Projection sur 2 composantes
    pca_projection = components[:, :2]
    eigenvalues_file = os.path.join(PCA_OUTPUT_DIR, 'eigenvalues.eval')
    with open(eigenvalues_file, 'w') as f:
        for eigenvalue in pca.explained_variance_:
            f.write(f'{eigenvalue}\n')
    eigenvectors = pca.components_
    eigenvectors_reshaped = eigenvectors.reshape(eigenvectors.shape[0], -1, 3)
    for i, eigenvector in enumerate(eigenvectors_reshaped):
        filename = os.path.join(PCA_OUTPUT_DIR, f'eigenvector_{i+1}.eig')
        np.savetxt(filename, eigenvector, fmt='%f')
    print("Résultats de la PCA sauvegardés dans " + PCA_OUTPUT_DIR)
    return pca_projection, pca.explained_variance_, eigenvalues_file



  ### 4. Métriques d'Erreur

In [ ]:
def compute_compactness(eigenvalues, threshold=0.95):
    total_variance = np.sum(eigenvalues)
    cumulative_variance = np.cumsum(eigenvalues) / total_variance
    num_components = int(np.argmax(cumulative_variance >= threshold) + 1)
    return num_components, cumulative_variance

def load_shapes(particles_dir, particle_type="world"):
    shapes = []
    for filename in os.listdir(particles_dir):
        if filename.endswith(particle_type + '.particles'):
            filepath = os.path.join(particles_dir, filename)
            shape = np.loadtxt(filepath)
            shapes.append(shape)
    return np.array(shapes)

def compute_specificity(real_shapes, num_particles, num_samples=1000):
    n, p, dim3 = real_shapes.shape
    d = p * dim3
    real_shapes = real_shapes.reshape(n, d)
    Y = real_shapes.T
    mu = np.mean(Y, axis=1, keepdims=True)
    Y_centered = Y - mu
    U, S, _ = np.linalg.svd(Y_centered, full_matrices=False)
    if S[0] < S[-1]:
        S = S[::-1]
        U = np.fliplr(U)
    specificities = np.zeros(n - 1)
    training_data = Y.T
    def shape_distance(ptsA, ptsB, pcount):
        A3 = ptsA.reshape(pcount, 3)
        B3 = ptsB.reshape(pcount, 3)
        return np.linalg.norm(A3 - B3, axis=1).sum()
    for m in tqdm(range(1, n), desc="Calcul de la spécificité"):
        epsi = U[:, :m]
        stdevs = np.sqrt(S[:m])
        betas = np.random.randn(m, num_samples)
        for i_mode in range(m):
            betas[i_mode, :] *= stdevs[i_mode]
        synthetic = epsi @ betas + mu
        min_dists = np.zeros(num_samples)
        for isyn in range(num_samples):
            synth_i = synthetic[:, isyn]
            best = 1e15
            for j in range(n):
                dist_ij = shape_distance(synth_i, training_data[j], num_particles)
                if dist_ij < best:
                    best = dist_ij
            min_dists[isyn] = best
        avg_min = np.mean(min_dists) / float(num_particles)
        specificities[m-1] = avg_min
    return specificities

def compute_generalization(real_shapes, num_particles):
    if len(real_shapes.shape) == 3 and real_shapes.shape[2] == 3:
        n, p, dim3 = real_shapes.shape
        d = p * dim3
        real_shapes = real_shapes.reshape(n, d)
    else:
        n, d = real_shapes.shape
    if d != 3 * num_particles:
        raise ValueError(f"d={d}, attendu 3*num_particles={3*num_particles}")
    P = real_shapes.T
    generalizations = np.zeros(n - 1)
    def shape_distance(ptsA, ptsB, pcount):
        A3 = ptsA.reshape(pcount, 3)
        B3 = ptsB.reshape(pcount, 3)
        return np.linalg.norm(A3 - B3, axis=1).sum()
    for m in range(1, n):
        total_dist = 0.0
        for leave in range(n):
            Y = np.zeros((d, n-1))
            Y[:, :leave] = P[:, :leave]
            Y[:, leave:] = P[:, leave+1:]
            mu = np.mean(Y, axis=1, keepdims=True)
            Y_centered = Y - mu
            U, S, _ = np.linalg.svd(Y_centered, full_matrices=False)
            epsi = U[:, :m]
            y_test = P[:, leave:leave+1]
            betas = epsi.T @ (y_test - mu)
            rec = epsi @ betas + mu
            dist = shape_distance(rec, y_test, num_particles) / float(num_particles)
            total_dist += dist
        generalizations[m - 1] = total_dist / float(n)
    return generalizations

def compute_error_metrics(particles_dir, PCA_OUTPUT_DIR, num_particles):
    print("\n----------------------------------------")
    print("Calcul des métriques d'erreur")
    real_shapes = load_shapes(particles_dir, "world")
    eigenvalues = np.loadtxt(os.path.join(PCA_OUTPUT_DIR, 'eigenvalues.eval'))
    req_comp, cum_variance = compute_compactness(eigenvalues)
    specifics = compute_specificity(real_shapes.reshape(real_shapes.shape[0], -1, 3), num_particles)
    generalizations = compute_generalization(real_shapes.reshape(real_shapes.shape[0], -1, 3), num_particles)
    metrics = {
        "compactness_required": req_comp,
        "cumulative_variance": cum_variance.tolist(),
        "specificity": specifics.tolist(),
        "generalization": generalizations.tolist()
    }
    return metrics



  ### 5. Pipeline

In [ ]:
def run_pipeline(run_params, run_index):
    """
    Pipeline complète :
      - Acquisition des données
      - Grooming
      - Rigid
      - Optimisation
      - PCA
      - Métriques
    """
    print("\n========================================")
    print(f"Lancement de la pipeline - Exécution {run_index}")
    overall_start = time.time()

    # Sorties
    BASE_OUTPUT = os.path.join(BASE_OUTPUT_DIR, f"Run_{run_index}")
    os.makedirs(BASE_OUTPUT, exist_ok=True)
    OUTPUT_PATH = os.path.join(BASE_OUTPUT, "OUTPUT")
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    groom_dir = os.path.join(OUTPUT_PATH, "groomed")
    PCA_OUTPUT_DIR = os.path.join(OUTPUT_PATH, "PCA_results")
    
    step_times = {}
    
    # 1: Acquisition
    t0 = time.time()
    shape_filenames, dataset_ids = acquire_data(DATASET_PATHS, SHAPE_EXT, OUTPUT_PATH)
    step_times["acquisition"] = time.time() - t0
    
    # 2: Grooming
    t0 = time.time()
    shape_seg_list, shape_names = groom_shapes(shape_filenames, dataset_ids, groom_dir)
    step_times["grooming"] = time.time() - t0
    
    # 3: Rigid
    t0 = time.time()
    rigid_transforms, groomed_files = rigid_transformations(shape_seg_list, shape_names, groom_dir)
    step_times["rigid"] = time.time() - t0
    
    # 4: Optimization
    t0 = time.time()
    OPT_PARAMS.update(run_params)
    particles_dir = optimize_particles(shape_seg_list, shape_filenames, rigid_transforms, groomed_files, OUTPUT_PATH)
    step_times["optimization"] = time.time() - t0
    
    # 5: PCA
    t0 = time.time()
    pca_projection, eigenvalues, _ = compute_pca(particles_dir, PCA_OUTPUT_DIR)
    step_times["pca"] = time.time() - t0
    
    # 6: Error metrics
    t0 = time.time()
    num_particles = OPT_PARAMS.get("number_of_particles", 128)
    metrics = compute_error_metrics(particles_dir, PCA_OUTPUT_DIR, num_particles)
    step_times["error_metrics"] = time.time() - t0
    
    overall_time = time.time() - overall_start
    print(f"Exécution {run_index} terminée en {overall_time:.2f} secondes")
    
    return {
        "pca_projection": pca_projection,
        "metrics": metrics,
        "params": run_params,
        "step_times": step_times,
        "total_execution_time": overall_time
    }



  ### 6. Génération du fichier Excel

In [ ]:
def _plot_metric_curve(data, title, ylabel, run_idx, outdir, figsize=(5, 3)):
    """
    Petite fonction utilitaire pour tracer et sauvegarder un graphique.
    - data: valeurs à tracer
    - title, ylabel: légendes du graphe
    - run_idx: index du run (pour nommer l'image)
    - outdir: répertoire de sauvegarde
    - figsize: tuple (width, height) en pouces
    """
    os.makedirs(outdir, exist_ok=True)

    filename = f"{title.replace(' ','_')}_Run_{run_idx}.png"
    image_path = os.path.join(outdir, filename)

    # Utilise figsize passé en paramètre
    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(range(1, len(data) + 1), data, marker='o')
    ax.set_title(title)
    ax.set_xlabel("Number of Modes")
    ax.set_ylabel(ylabel)
    ax.grid(True)

    plt.savefig(image_path, dpi=130, bbox_inches="tight")
    plt.close(fig)

    return image_path


In [ ]:
def save_results_to_excel(results, excel_filename, grid_keys):
    """
    Feuille "PARAMS" :
      - Colonnes A–H : (Run Index, grid_keys, 4 temps)
      - Largeur des colonnes A–G = 15, I = 30
      - En dessous (ligne len(results)+3), colonnes I–J : 
        tous les paramètres OPT_PARAMS + grooming (multiligne).
    
    Feuille "Run_i" :
      - Graphes (Compactness, Specificity, Generalization) aux lignes 1, 13, 25,
      - Taille 280×168 px,
      - Infos 8 lignes en dessous,
      - PC1/PC2 à la ligne 36.
    """

    print("\nSauvegarde des résultats dans le fichier Excel...")

    # 1) Tableau principal A–H : run index, grid keys, 4 temps
    table_cols = ["Run Index"] + list(grid_keys) + [
        "time_grooming", "time_rigid", "time_optimization", "time_total"
    ]
    data_for_df = []

    for i, res in enumerate(results, start=1):
        row_dict = {}
        row_dict["Run Index"] = i

        # Paramètres variables
        for k in grid_keys:
            row_dict[k] = res["params"].get(k, None)

        # 4 temps en minutes arrondies
        st = res.get("step_times", {})
        row_dict["time_grooming"]     = round(st.get("grooming", 0) / 60)
        row_dict["time_rigid"]        = round(st.get("rigid", 0) / 60)
        row_dict["time_optimization"] = round(st.get("optimization", 0) / 60)
        row_dict["time_total"]        = round(res.get("total_execution_time", 0) / 60)

        data_for_df.append(row_dict)

    params_df = pd.DataFrame(data_for_df, columns=table_cols)

    # 2) Liste de tous les paramètres
    grooming_keys = [
        "ANTIALIAS_ITERATIONS", "ISO_SPACING", "PAD_SIZE",
        "PAD_VALUE", "ISO_VALUE", "ICP_ITERATIONS"
    ]
    all_param_names = sorted(set(OPT_PARAMS.keys()) | set(grooming_keys))

    # 3) Écriture du DataFrame dans "PARAMS"
    writer = pd.ExcelWriter(excel_filename, engine='openpyxl')
    params_df.to_excel(writer, sheet_name="PARAMS", index=False)

    ws_params = writer.book["PARAMS"]

    # --- Ajuster la largeur des colonnes ---
    for col_letter in ["A","B","C","D","E","F","G"]:
        ws_params.column_dimensions[col_letter].width = 15
    # Colonne H n'est pas explicitement mentionnée, on peut lui laisser la largeur par défaut
    # ou la mettre aussi à 15 si souhaité :
    ws_params.column_dimensions["H"].width = 15
    ws_params.column_dimensions["I"].width = 30
    # La J reste par défaut

    # 4) Insérer dans la feuille "PARAMS", en I–J (ligne = len(results)+3),
    #    tous les paramètres en multiligne (noms / valeurs)
    from openpyxl.styles import Alignment

    row_params = len(results) + 3
    param_names_list  = []
    param_values_list = []

    for pname in all_param_names:
        if pname in OPT_PARAMS:
            val = OPT_PARAMS[pname]
        else:
            if pname == "ANTIALIAS_ITERATIONS":
                val = ANTIALIAS_ITERATIONS
            elif pname == "ISO_SPACING":
                val = str(ISO_SPACING)
            elif pname == "PAD_SIZE":
                val = PAD_SIZE
            elif pname == "PAD_VALUE":
                val = PAD_VALUE
            elif pname == "ISO_VALUE":
                val = ISO_VALUE
            elif pname == "ICP_ITERATIONS":
                val = ICP_ITERATIONS
            else:
                val = None

        param_names_list.append(str(pname))
        param_values_list.append(str(val))

    multiline_names  = "\n".join(param_names_list)
    multiline_values = "\n".join(param_values_list)

    c_names  = ws_params.cell(row=row_params, column=9)   # I
    c_values = ws_params.cell(row=row_params, column=10)  # J

    c_names.value  = multiline_names
    c_values.value = multiline_values

    c_names.alignment  = Alignment(wrap_text=True)
    c_values.alignment = Alignment(wrap_text=True)

    # 5) Feuilles "Run_i"
    for i, res in enumerate(results, start=1):
        sheet_name = f"Run_{i}"
        dummy_df = pd.DataFrame()
        dummy_df.to_excel(writer, sheet_name=sheet_name, index=False)
        ws_run = writer.book[sheet_name]

        # Graphes
        plot_dir = os.path.join(BASE_OUTPUT_DIR, f"Run_{i}", "plots")
        os.makedirs(plot_dir, exist_ok=True)

        cvar = res["metrics"]["cumulative_variance"]
        specificity_data = res["metrics"]["specificity"]
        general_data = res["metrics"]["generalization"]

        compactness_img = _plot_metric_curve(cvar, "Compactness", "Variance", i, plot_dir)
        specificity_img = _plot_metric_curve(specificity_data, "Specificity Error", "Error", i, plot_dir)
        general_img     = _plot_metric_curve(general_data, "Generalization Error", "Error", i, plot_dir)

        from openpyxl.drawing.image import Image as ExcelImage

        # Placement plus serré
        row_img1 = 1
        row_img2 = 12
        row_img3 = 23

        # Agrandir de 40 % => 280×168
        imgA = ExcelImage(compactness_img)
        imgA.width, imgA.height = 310, 180

        imgB = ExcelImage(specificity_img)
        imgB.width, imgB.height = 310, 180

        imgC = ExcelImage(general_img)
        imgC.width, imgC.height = 310, 180

        ws_run.add_image(imgA, f"A{row_img1}")
        ws_run.add_image(imgB, f"A{row_img2}")
        ws_run.add_image(imgC, f"A{row_img3}")

        comp_req = res["metrics"]["compactness_required"]
        ws_run.cell(row=row_img1+9, column=1, value=f"Composantes pour 95%: {comp_req}")

        if len(specificity_data) > 0:
            spec_final = specificity_data[-1]
            ws_run.cell(row=row_img2+9, column=1,
                        value=f"Specificity final error: {spec_final:.4f}")

        if len(general_data) > 0:
            gen_final = general_data[-1]
            ws_run.cell(row=row_img3+9, column=1,
                        value=f"Generalization final error: {gen_final:.4f}")

        # PC1/PC2 à la ligne 36
        pc1_pc2_df = pd.DataFrame(res["pca_projection"], columns=["PC1", "PC2"])
        pc1_pc2_df.to_excel(writer, sheet_name=sheet_name, startrow=34, index=False)

    # Sauvegarde
    writer.close()
    print(f"Fichier Excel sauvegardé : {excel_filename}")



  ### 7. Fonction Principale

In [ ]:
grid_keys = list(GRID_PARAMS.keys())
grid_values = [GRID_PARAMS[k] for k in grid_keys]
runs = []

for combination in itertools.product(*grid_values):
    run_param = {}
    for key, value in zip(grid_keys, combination):
        run_param[key] = value
    runs.append(run_param)

results = []


In [ ]:
for i, run in enumerate(runs, start=1):
    print("\n========================================")
    print("Exécution avec les paramètres :")
    for key, value in run.items():
        print(f"  {key} = {value}")
    res = run_pipeline(run, i)
    results.append(res)


In [ ]:
excel_filename = os.path.join(BASE_OUTPUT_DIR, "grid_search_results.xlsx")
os.makedirs(BASE_OUTPUT_DIR, exist_ok=True)
save_results_to_excel(results, excel_filename, grid_keys)

